In [2]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [3]:
df = pd.read_csv('udemy_course_data.csv')

In [4]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13


In [5]:
df.shape

(3683, 18)

In [6]:
def pre_process(text):

    text = text.lower()
    ## remove punctuation
    pun = string.punctuation
    pun = string.punctuation
    text = text.translate(str.maketrans(" "," ",pun))

    text = text.split(" ")
    ## remove stop words
    stop_words = stopwords.words("english")
    text = [word for word in text if word not in stop_words]

   



    return " ".join(text)

In [7]:
df['course_title'] = df['course_title'].apply(lambda x:pre_process(x))

## vectorise the clean title


In [8]:
cv = CountVectorizer()
cvmat = cv.fit_transform(df['course_title'])

In [9]:
cvmat.toarray().shape

(3683, 3743)

## cosin similarity

In [10]:
cos_sim = cosine_similarity(cvmat)

In [11]:
cos_sim

array([[1.        , 0.18898224, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18898224, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.37796447],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.37796447, 0.        ,
        1.        ]])

## Recommend course

In [12]:
text = df['course_title'][0]

In [13]:
#get the specific index of a title
course_index = pd.Series(index=df['course_title'],data=df.index).drop_duplicates()
index = course_index[text]



In [14]:
enumerated_scores = list(enumerate(cos_sim[index]))

In [15]:
sorted_scores = sorted(enumerated_scores, key=lambda x: x[1], reverse=True)[1:]

In [16]:
sorted_index = [i[0] for i in sorted_scores]
sorted_score = [i[1] for i in sorted_scores]

In [17]:
temp_df = df.iloc[sorted_index]

In [18]:
temp_df['similarity_score'] = sorted_score

C:\Users\Tafique Hossain Khan\AppData\Local\Temp\ipykernel_2792\1180948622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['similarity_score'] = sorted_score


In [19]:
temp_df.head(1)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,similarity_score
39,965832,complete investment banking course 2017,https://www.udemy.com/the-complete-investment-...,True,195,8575,809,110,All Levels,5.5 hours,2016-10-10T16:16:26Z,Business Finance,1672125,2016-10-10,16:16:26Z,2016,10,10,0.67082


In [20]:
final_recommended_course = temp_df[['course_title','url','similarity_score','price']]

In [21]:
final_recommended_course[final_recommended_course['similarity_score']>=0.5]

,course_title,url,similarity_score,price
39,complete investment banking course 2017,https://www.udemy.com/the-complete-investment-...,0.67082,195
3479,ultimate jquery course,https://www.udemy.com/essential-jquery-training/,0.57735,20
242,advanced accounting investment banking,https://www.udemy.com/advanced-accounting-for-...,0.50000,50
419,investment banking recruitment series,https://www.udemy.com/investmentbanking/,0.50000,40
2719,ultimate web development course,https://www.udemy.com/build-an-instant-update-...,0.50000,50
2807,ultimate wordpress plugin course,https://www.udemy.com/wordpress-plugin-course/,0.50000,100


In [22]:
final_recommended_course

,course_title,url,similarity_score,price
39,complete investment banking course 2017,https://www.udemy.com/the-complete-investment-...,0.67082,195
3479,ultimate jquery course,https://www.udemy.com/essential-jquery-training/,0.57735,20
242,advanced accounting investment banking,https://www.udemy.com/advanced-accounting-for-...,0.50000,50
419,investment banking recruitment series,https://www.udemy.com/investmentbanking/,0.50000,40
2719,ultimate web development course,https://www.udemy.com/build-an-instant-update-...,0.50000,50
...,...,...,...,...
3678,learn jquery scratch master javascript library,https://www.udemy.com/easy-jquery-for-beginner...,0.00000,100
3679,design wordpress website coding,https://www.udemy.com/how-to-make-a-wordpress-...,0.00000,25
3680,learn build using polymer,https://www.udemy.com/learn-and-build-using-po...,0.00000,40
3681,css animations create amazing effects website,https://www.udemy.com/css-animations-create-am...,0.00000,50


In [23]:
def recommend_course(df,no_of_course,text):
    #get the specific index of a title
    course_index = pd.Series(index=df['course_title'],data=df.index).drop_duplicates()
    index = course_index[text]
    enumerated_scores = list(enumerate(cos_sim[index]))
    sorted_scores = sorted(enumerated_scores, key=lambda x: x[1], reverse=True)[1:]

    sorted_index = [i[0] for i in sorted_scores]
    sorted_score = [i[1] for i in sorted_scores]
    temp_df = df.iloc[sorted_index]

    temp_df['similarity_score'] = sorted_score

    final_recommended_course = temp_df[['course_title','url','similarity_score','price']]
    final_recommended_course = final_recommended_course[final_recommended_course['similarity_score']>=0.5].head(no_of_course)
    return final_recommended_course

In [24]:
df['course_title'][0]

'ultimate investment banking course'

In [25]:
ans = recommend_course(df,6,'non directional weekly options trading system  etf  emini')

C:\Users\Tafique Hossain Khan\AppData\Local\Temp\ipykernel_2792\369221961.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['similarity_score'] = sorted_score


In [26]:
df['url'][89]

'https://www.udemy.com/emini-es-futures-advanced-non-directional-options-trading/'

In [27]:
ans

,course_title,url,similarity_score,price
89,emini es futures advanced non directional opt...,https://www.udemy.com/emini-es-futures-advance...,0.625000,200
43,options trading win weekly options,https://www.udemy.com/work-from-home-setup-you...,0.534522,115
514,winning options trading system,https://www.udemy.com/winning-options-trading-...,0.530330,105
991,simple options trading system,https://www.udemy.com/safe-and-easy-options-tr...,0.530330,35


In [28]:
list(ans['course_title'])

['emini es futures  advanced non directional options trading',
 'options trading  win weekly options',
 'winning options trading system',
 'simple options trading system']

In [29]:
list(ans['url'])

['https://www.udemy.com/emini-es-futures-advanced-non-directional-options-trading/',
 'https://www.udemy.com/work-from-home-setup-your-own-options-trading-business/',
 'https://www.udemy.com/winning-options-trading-system/',
 'https://www.udemy.com/safe-and-easy-options-trading/']

In [30]:
dict(zip(list(ans['course_title']),list(ans['url'])))

{'emini es futures  advanced non directional options trading': 'https://www.udemy.com/emini-es-futures-advanced-non-directional-options-trading/',
 'options trading  win weekly options': 'https://www.udemy.com/work-from-home-setup-your-own-options-trading-business/',
 'winning options trading system': 'https://www.udemy.com/winning-options-trading-system/',
 'simple options trading system': 'https://www.udemy.com/safe-and-easy-options-trading/'}

## recommending courese based on the keyword

In [66]:
result_df = df[df['course_title'].str.contains('python')].sort_values(by='num_reviews',ascending=False).head(6)

In [67]:
result_df['course_title']

3205      python django full stack web developer bootcamp
3143    complete python web course build 8 python web ...
2575             coding entrepreneurs learn python django
3208                               rest apis flask python
3165      professional python web development using flask
3512           fun creative web engineering python web2py
Name: course_title, dtype: object

In [72]:
import preprocessing

In [79]:
course_title,course_url = preprocessing.extract_feature(result_df)

AttributeError: module 'preprocessing' has no attribute 'extract_feature'

In [74]:
course_title

['https://www.udemy.com/python-and-django-full-stack-web-developer-bootcamp/',
 'https://www.udemy.com/the-complete-python-web-course-learn-by-building-8-apps/',
 'https://www.udemy.com/coding-for-entrepreneurs/',
 'https://www.udemy.com/rest-api-flask-and-python/',
 'https://www.udemy.com/python-flask-course/',
 'https://www.udemy.com/web-engineering-with-python-and-web2py/']

In [75]:
course_url

['python django full stack web developer bootcamp',
 'complete python web course build 8 python web apps',
 'coding entrepreneurs learn python django',
 'rest apis flask python',
 'professional python web development using flask',
 'fun creative web engineering python web2py']